In [ ]:
import numpy as np
from ctapipe.io import event_source
from ctapipe.io import EventSeeker
import matplotlib.pyplot as plt
import numpy as np
from ctapipe.instrument import CameraGeometry
from ctapipe.visualization import CameraDisplay

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.rcParams['font.size'] = 20


In [ ]:
def format_axes(ax):
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.tick_params(
    axis='x',       # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
    ax.tick_params(
    axis='y',       # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off
    
    return ax

In [ ]:
source = event_source(input_url="../../data/real_data/LST-1.1.Run01627.0001.fits.fz",
                      max_events=100)
subarray = source.subarray
tel_id = 1
geom = subarray.tels[tel_id].camera.geometry

In [ ]:
for i, ev in enumerate(source):
    n_pixels = geom.n_pixels

    std_signal = np.zeros(n_pixels)
    for pixel in range(0, n_pixels):
        std_signal[pixel] = np.max(ev.r0.tel[tel_id].waveform[0, pixel, 2:38])

    if((np.size(std_signal[std_signal>1000.]) < 15) or (np.size(std_signal[std_signal>1000.]) > 1800)):
        continue  
    print(f"Event {ev.lst.tel[0].evt.event_id}, Max: {np.max(std_signal)} counts")


    fig, ax = plt.subplots(figsize=(10,10))
    disp0 = CameraDisplay(geom, ax=ax)
    disp0.cmap = 'viridis'
    disp0.image = std_signal
    disp0.add_colorbar(ax=ax)

    # Establish max and min
    max_color = np.max(std_signal)
    min_color = np.min(std_signal)

    disp0.set_limits_minmax(min_color, max_color)
    ax.set_title(f"Event {ev.lst.tel[tel_id].evt.event_id}")
    format_axes(ax)
#     fig.savefig("Images_LST/Event_%i.png"%(ev.lst.tel[0].evt.event_id))

    plt.show()

In [ ]:
# If you want to make a movie with all the slices

max_color = np.max(std_signal)
min_color = np.min(std_signal)

!mkdir -p image_test


for cell in range(1,39):
    print("cell",cell)
    fig, ax = plt.subplots(figsize=(10,10))
    disp0 = CameraDisplay(geom, ax=ax)
    disp0.cmap = 'viridis'
    disp0.add_colorbar(ax=ax)
    disp0.image = ev.r0.tel[tel_id].waveform[0,:,cell]
    disp0.set_limits_minmax(min_color, max_color)
    format_axes(ax)
    ax.set_title(f"Event {ev.lst.tel[tel_id].evt.event_id}, Time {cell} ns")
    fig.savefig("image_test/Event_{:02d}_cell{:02d}.png".format(ev.lst.tel[tel_id].evt.event_id,cell))
    #plt.show()

In [ ]:
# make a gif movie
!convert image_test/'Event_{ev.lst.tel[tel_id].evt.event_id}*png' image_test/'Event_{ev.lst.tel[tel_id].evt.event_id}.gif'